# Table-Valued Function Examples 2: Arrow Support & Deregistration

This notebook demonstrates advanced TVF features including Arrow return types and function deregistration.

In [ ]:
import duckdb
import pandas as pd
import random
from datetime import datetime, timedelta

## Example 1: Arrow Table Return Type (Placeholder)

Demonstrate the Arrow table return type placeholder functionality.

In [ ]:
def generate_arrow_data(count: int = 5):
    """Generate data that would eventually return as Arrow table."""
    data = []
    for i in range(count):
        data.append((
            f"record_{i}",
            random.randint(1, 100),
            random.uniform(10.0, 1000.0),
            random.choice([True, False])
        ))
    return data

conn = duckdb.connect()

# Define schema for Arrow table
schema = [
    ("name", "VARCHAR"),
    ("value", "INTEGER"),
    ("price", "DOUBLE"),
    ("active", "BOOLEAN")
]

try:
    # Register TVF with arrow_table return type
    tvf = conn.create_table_function(
        name="arrow_table_test",
        callable=generate_arrow_data,
        parameters=None,
        schema=schema,
        return_type="arrow_table"
    )
    
    print("✓ Arrow table TVF registered successfully")
    
    # Try to use it - should show placeholder message
    result = conn.execute("SELECT * FROM arrow_table_test(3)").fetchall()
    print(f"Unexpected success: {result}")
    
except Exception as e:
    print(f"✓ Expected placeholder error: {e}")

conn.close()

## Example 2: Arrow Batch Return Type (Placeholder)

Demonstrate the Arrow batch return type placeholder functionality.

In [ ]:
def generate_batch_data(batch_size: int = 10):
    """Generate data that would eventually return as Arrow batches."""
    return [(f"batch_{i}", i * 2, f"group_{i % 3}") for i in range(batch_size)]

conn = duckdb.connect()

schema = [
    ("id", "VARCHAR"),
    ("value", "INTEGER"),
    ("group_name", "VARCHAR")
]

try:
    # Register TVF with arrow_batch return type
    tvf = conn.create_table_function(
        name="arrow_batch_test",
        callable=generate_batch_data,
        parameters=None,
        schema=schema,
        return_type="arrow_batch"
    )
    
    print("✓ Arrow batch TVF registered successfully")
    
    # Try to use it - should show placeholder message
    result = conn.execute("SELECT * FROM arrow_batch_test(5)").fetchall()
    print(f"Unexpected success: {result}")
    
except Exception as e:
    print(f"✓ Expected placeholder error: {e}")

conn.close()

## Example 3: TVF Deregistration

Demonstrate how to register and then unregister table-valued functions.

In [ ]:
def temporary_function(count: int = 3):
    """A function that we will register and then remove."""
    return [(f"temp_{i}", i + 100) for i in range(count)]

conn = duckdb.connect()

schema = [("name", "VARCHAR"), ("value", "INTEGER")]

print("=== Testing TVF Deregistration ===")

# Step 1: Register the function
tvf = conn.create_table_function(
    name="temporary_function",
    callable=temporary_function,
    parameters=None,
    schema=schema,
    return_type="records"
)

print("1. ✓ TVF registered successfully")

# Step 2: Use the function to verify it works
try:
    result = conn.execute("SELECT * FROM temporary_function(2)").fetchall()
    print(f"2. ✓ TVF works before deregistration: {result}")
except Exception as e:
    print(f"2. ✗ Unexpected error before deregistration: {e}")

# Step 3: Deregister the function
conn.unregister_table_function("temporary_function")
print("3. ✓ TVF unregistered successfully")

# Step 4: Try to use the function after deregistration - should fail
try:
    result = conn.execute("SELECT * FROM temporary_function(2)").fetchall()
    print(f"4. ✗ Function should not be available: {result}")
except Exception as e:
    print(f"4. ✓ Expected error after deregistration: {e}")

conn.close()
print("\n=== Deregistration Test Completed ===")

## Example 4: Multiple Function Registration and Selective Deregistration

Register multiple functions and show that deregistration only affects the target function.

In [ ]:
def function_a(count: int = 2):
    return [(f"a_{i}", i) for i in range(count)]

def function_b(count: int = 2):
    return [(f"b_{i}", i * 10) for i in range(count)]

def function_c(count: int = 2):
    return [(f"c_{i}", i * 100) for i in range(count)]

conn = duckdb.connect()
schema = [("name", "VARCHAR"), ("value", "INTEGER")]

print("=== Testing Selective Deregistration ===")

# Register all three functions
for name, func in [("func_a", function_a), ("func_b", function_b), ("func_c", function_c)]:
    conn.create_table_function(
        name=name,
        callable=func,
        schema=schema,
        return_type="records"
    )

print("1. ✓ All three functions registered")

# Test all functions work
results = {}
for func_name in ["func_a", "func_b", "func_c"]:
    try:
        result = conn.execute(f"SELECT * FROM {func_name}(2)").fetchall()
        results[func_name] = result
        print(f"2. ✓ {func_name}: {result}")
    except Exception as e:
        print(f"2. ✗ {func_name} failed: {e}")

# Deregister only function_b
conn.unregister_table_function("func_b")
print("3. ✓ func_b deregistered")

# Test that func_a and func_c still work, but func_b doesn't
print("4. Testing after selective deregistration:")
for func_name in ["func_a", "func_b", "func_c"]:
    try:
        result = conn.execute(f"SELECT * FROM {func_name}(2)").fetchall()
        if func_name == "func_b":
            print(f"   ✗ {func_name} should have failed but returned: {result}")
        else:
            print(f"   ✓ {func_name} still works: {result}")
    except Exception as e:
        if func_name == "func_b":
            print(f"   ✓ {func_name} correctly failed: {e}")
        else:
            print(f"   ✗ {func_name} should still work: {e}")

conn.close()
print("\n=== Selective Deregistration Test Completed ===")

## Example 5: Arrow Return Type Variations

Test different ways to specify Arrow return types.

In [ ]:
import pyarrow as pa


def sample_data(n: int = 5) -> pa.Table:
    schema = pa.schema([
        ("col1", pa.string()),
        ("col2", pa.int32()),
        ("col3", pa.float64())
    ])
    data = {
        "col1": [f"row{i}" for i in range(n)],
        "col2": list(range(n)),
        "col3": [float(i) for i in range(n)]
    }
    return pa.Table.from_pydict(data, schema=schema)

schema = [("col1", "VARCHAR"), ("col2", "INTEGER"), ("col3", "DOUBLE")]

print("=== Testing Arrow Return Type Variations ===")

# Test different arrow type specifications
arrow_types = [
    ("arrow_table", "arrow_table_func"),
    ("arrow", "arrow_func"),  # Should be treated same as arrow_table
    ("arrow_batch", "arrow_batch_func")
]

for return_type, func_name in arrow_types:
    with duckdb.connect() as conn:

        try:
            # Register function
            conn.create_table_function(
                name=func_name,
                callable=sample_data,
                schema=schema,
                return_type=return_type
            )
            print(f"✓ {return_type} function registered as {func_name}")
            
            # Try to use it
            result = conn.execute(f"SELECT * FROM {func_name}()").fetchall()
            print(f"✗ {return_type} should have shown placeholder error")
            
        except Exception as e:
            if "not yet implemented" in str(e):
                print(f"✓ {return_type} correctly shows placeholder: {e}")
            else:
                print(f"✗ {return_type} unexpected error: {e}")
        
    print()

print("=== Arrow Type Variations Test Completed ===")

## Summary

This notebook demonstrates:

1. **Arrow Table Support**: `arrow_table` and `arrow` return types are recognized and show appropriate placeholder messages
2. **Arrow Batch Support**: `arrow_batch` return type is recognized with placeholder functionality
3. **Function Deregistration**: `unregister_table_function()` properly removes functions from all registries
4. **Selective Deregistration**: Only the target function is removed, others remain functional
5. **Return Type Variations**: Different arrow type specifications work correctly

All Arrow return types are properly supported with placeholder implementations that can be extended when full Arrow support is added.